In [3]:
import numpy as np
import seaborn as sns
import pandas as pd
import random
import pyswarms as ps

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
data_inputs = pd.read_csv('Dataset/arrhythmia.data',sep = ',',header=None)
data_inputs.replace("?", np.nan, inplace= True)
median_value=data_inputs[12].median()
data_inputs[12]=data_inputs[12].fillna(median_value)
median_value=data_inputs[14].median()
data_inputs[14]=data_inputs[14].fillna(median_value)
median_value=data_inputs[10].median()
data_inputs[10]=data_inputs[10].fillna(median_value)
median_value=data_inputs[11].median()
data_inputs[11]=data_inputs[11].fillna(median_value)
median_value=data_inputs[13].median()
data_inputs[13]=data_inputs[13].fillna(median_value)
y = data_inputs.iloc[:,-1]
y = y.values
X = data_inputs.drop(279,axis=1)
X = X.values


In [4]:
from sklearn import linear_model

# Create an instance of the classifier
classifier = linear_model.LogisticRegression(solver='liblinear',multi_class='ovr')

# Define objective function
def f_per_particle(m, alpha):
    """Computes for the objective function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.
    alpha: float (default is 0.5)
        Constant weight for trading-off classifier performance
        and number of features

    Returns
    -------
    numpy.ndarray
        Computed objective function
    """
    total_features = 279
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X
    else:
        X_subset = X[:,m==1]
    # Perform classification and store performance in P
    classifier.fit(X_subset, y)
    P = (classifier.predict(X_subset) == y).mean()
    # Compute for the objective function
    j = (alpha * (1.0 - P) + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))

    return j

0.9473994165763411

In [20]:
def getK(D):
    K = []
    for k in range(D):
        s = 1
        for i in range(k):
            s += (D-i)
        l = ((D-k)/s)
        K.append(l)
    K[0] = 0.95
    return K

In [22]:
k = getK(10)

In [48]:
np.random.choice(list(range(10)), 1, k).item(0)+1

6

In [41]:
v.item(0)

6

In [112]:
def initialize(D, NoP):
    pk = getK(D)
    k = np.random.choice(list(range(D)), 1, pk).item(0)+1
    X=np.zeros((NoP, k))
    V=np.zeros((NoP, k))
    C=np.zeros((NoP, k))
    for i in range(NoP):
        for j in range(k):
            X[i][j] = random.random()
            V[i][j] = random.uniform(0,1)
    cor = [0]*NoP
    for i in range(NoP):
        for j in range(k):
            Xim = np.mean(X[i])
            Xjm = np.mean(X[j])
            num = (np.sum(X[i])-Xim*(k-1))*(np.sum(X[j])-Xjm*(k-1))
            den = ((np.sum(X[i])-Xim*(k-1))**2) * ((np.sum(X[j])-Xjm*(k-1))**2)
            C[i][j] = num/den
        #s = np.sum(C[i])
        #for j in range(k):
        #    C[i][j] /= s
        if(i != j):
            cor[i] = np.sum(C[i])/k-1
    s = np.sum(cor)
    cor /= s
    return cor

In [115]:
initialize(8, 6)

array([0.21696647, 0.22027219, 0.1379736 , 0.        , 0.21306519,
       0.21172255])

In [5]:
def f(x, alpha=0.88):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i], alpha) for i in range(n_particles)]
    return np.array(j)

In [6]:
%%time
# Initialize swarm, arbitrary
options = {'c1': 0.5, 'c2': 0.5, 'w':0.9, 'k': 30, 'p':2}

# Call instance of PSO
dimensions = 279 # dimensions should be the number of features
optimizer = ps.discrete.BinaryPSO(n_particles=30, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=100)

2019-10-16 14:12:31,871 - pyswarms.discrete.binary - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.5, 'w': 0.9, 'k': 30, 'p': 2}
pyswarms.discrete.binary: 100%|██████████████████████████████████████████████████████████████|100/100, best_cost=0.0542
2019-10-16 14:34:43,783 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.05423922352269484, best pos: [1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0
 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 1 0 1 0 0
 1 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1
 0 1 1 1 1 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0
 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1
 0 0 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1
 0 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1]


Wall time: 22min 11s


In [7]:
# Create two instances of LogisticRegression
classfier = linear_model.LogisticRegression()

# Get the selected features from the final positions
X_selected_features = X[:,pos==1]  # subset

# Perform classification and store performance in P
classifier.fit(X_selected_features, y)

# Compute performance
subset_performance = (classifier.predict(X_selected_features) == y).mean()


print('Subset performance: %.3f' % (subset_performance))

Subset performance: 0.991
